<a href="https://colab.research.google.com/github/akvo/oak-india/blob/main/scripts/hh_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Load Data

import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import folium
from folium import plugins
import numpy as np
import branca.colormap as cm
import json
import plotly.graph_objects as go
from scipy import stats
import matplotlib.colors as mcolors
import requests
import altair as alt
import warnings

geojson_data=requests.get('https://raw.githubusercontent.com/akvo/oak-india/refs/heads/main/data/villages.geojson').json()
csv_files = {
    'HH - Gosaba 1': 'https://raw.githubusercontent.com/akvo/oak-india/refs/heads/main/data/hh-gosaba-1-cleaned.csv',
    'HH - Gosaba 2': 'https://raw.githubusercontent.com/akvo/oak-india/refs/heads/main/data/hh-gosaba-2-cleaned.csv',
    'HH - Pathar': 'https://raw.githubusercontent.com/akvo/oak-india/refs/heads/main/data/hh-pathar.csv',
    # Add more CSV files as needed
}
df = pd.DataFrame()  # Initialize an empty DataFrame

# Create a dropdown for selecting CSV files
csv_dropdown = widgets.Dropdown(
    options=list(csv_files.keys()),
    description='Select CSV:',
)
load_button = widgets.Button(description="Load Data")
def load_csv(b):
    global df
    selected_file = csv_files[csv_dropdown.value]
    df=pd.read_csv(selected_file,encoding='utf-8',low_memory=False)
    df.columns = df.columns.str.replace(':', '', regex=False)
    columns_to_concatenate = [
    'গ্রামের নাম',
    'পাড়ার নাম',
    'উত্তরদাতার নাম(দিদির নাম)',
    'তথ্য সংগ্রহকারীর নাম'
      ]

    # Create the 'loc-res-sub' column safely
    df['loc-res-sub'] = df.apply(lambda row: '$'.join(
        str(row[col]) if col in row.index and pd.notna(row[col]) else '' for col in columns_to_concatenate
    ), axis=1)

    print(f"Loaded {csv_dropdown.value}")
    print(f"Number of rows: {len(df)}")

load_button.on_click(load_csv)

# Display the dropdown and button
display(csv_dropdown, load_button)

Dropdown(description='Select CSV:', options=('HH - Gosaba 1', 'HH - Gosaba 2', 'HH - Pathar'), value='HH - Gos…

Button(description='Load Data', style=ButtonStyle())

<ipython-input-1-069e54b598d1>:37: DtypeWarning: Columns (71) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(selected_file,encoding='utf-8')


Loaded HH - Pathar
Number of rows: 11831


In [2]:
# @title Filter data

# Function to update the map based on dropdown selections
def update_map(cat_col, selected_values):
    if selected_values:
        print(cat_col,selected_values)
        filtered_df = df[df[cat_col].isin(selected_values)]
    else:
        filtered_df = df  # Show all if no selection
    return filtered_df


# Create dropdowns for categorical columns
cat_col_dropdown = widgets.Dropdown(
    options=df.select_dtypes(include=['object','period[M]']).columns.tolist(),
    description='Column:',
)

# Create a multi-select dropdown for unique values
value_dropdown = widgets.SelectMultiple(
    options=df[cat_col_dropdown.value].unique().tolist(),
    description='Values:',
)

# Function to update the unique values dropdown based on selected column
def update_value_dropdown(change):
    selected_col = change['new']
    value_counts = df[selected_col].value_counts()
    # Create a sorted list of options with counts
    sorted_values = sorted(value_counts.items(), key=lambda x: x[1], reverse=True)
    # Format the options to include counts
    value_dropdown.options = [f"{val} (Count: {count})" for val, count in sorted_values]
    value_dropdown.value = []  # Reset selected values

# Link the column dropdown to the update function
cat_col_dropdown.observe(update_value_dropdown, names='value')

# Create a button to update the map
update_button = widgets.Button(description="Filter Data")

# Link the button to the update function
def on_button_click(b):
    global df
    selected_values = [val.split(" (Count: ")[0] for val in value_dropdown.value]  # Extract original values
    if selected_values:
        df = df[df[cat_col_dropdown.value].isin(selected_values)]
        print(f"Filtered data based on {cat_col_dropdown.value}: {selected_values}")
    else:
        print("No filter applied.")


update_button.on_click(on_button_click)

# Display the dropdowns and button
display(cat_col_dropdown, value_dropdown, update_button)


Dropdown(description='Column:', options=('start', 'end', 'ব্লক এর নাম', 'গ্রাম পঞ্চায়েত', 'গ্রামের নাম', 'পাড়…

SelectMultiple(description='Values:', options=('2024-06-02T00:04:00Z', '2024-06-02T11:50:11Z', '2024-06-02T00:…

Button(description='Filter Data', style=ButtonStyle())

Filtered data based on গ্রামের নাম: ['Banashyam nagar']
Filtered data based on পাড়ার নাম:: ['Bidhan palli']


In [3]:
# @title Create Map


def rgba_to_hex(rgba):
    """Convert RGBA tuple to hex color string."""
    return mcolors.to_hex(rgba)
exclude_cols = ['_বাড়ির অবস্থান _latitude', '_বাড়ির অবস্থান _longitude','_বাড়ির অবস্থান _precision', '_বাড়ির অবস্থান _altitude',
                '_id', '_uuid', '_submission_time', '_validation_status',
                '_notes', '_status', '_submitted_by', '__version__', '_tags', '_index','start','end']
plot_columns = [col for col in df.columns if col not in exclude_cols]
output = widgets.Output()

def create_color_map(data):
    """Create a color map based on data type"""
    if pd.api.types.is_numeric_dtype(data):
        # Calculate quartiles and IQR
        Q1 = data.quantile(0.25)
        Q3 = data.quantile(0.75)
        IQR = Q3 - Q1

        # Define outlier boundaries
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Create bounds using a combination of outlier boundaries and percentiles
        bounds = [
            data.min(),  # Minimum value
            lower_bound,  # Lower outlier boundary
            Q1,          # 25th percentile
            data.median(),  # Median
            Q3,          # 75th percentile
            upper_bound, # Upper outlier boundary
            data.max()   # Maximum value
        ]
        bounds = sorted(list(set([round(b, 2) for b in bounds])))  # Remove duplicates and round

        # Create colormap with more distinct colors
        colormap = cm.StepColormap(
            colors=['#313695', '#4575b4', '#74add1', '#abd9e9', '#fdae61', '#f46d43', '#d73027'],
            vmin=bounds[0],
            vmax=bounds[-1],
            index=bounds,
            caption=(f'Min: {bounds[0]:.2f} | '
                    f'Q1: {Q1:.2f} | '
                    f'Median: {data.median():.2f} | '
                    f'Q3: {Q3:.2f} | '
                    f'Max: {bounds[-1]:.2f}\n'
                    f'(IQR: {IQR:.2f})')
        )
        return colormap
    else:
        # For categorical data, create a categorical color map
        unique_values = data.value_counts()
        if len(unique_values) <= 20:
            top_categories = unique_values
        else:
            top_categories = unique_values.head(20)

        distinct_colors = [
            '#e6194B', '#3cb44b', '#4363d8', '#f58231', '#911eb4',
            '#42d4f4', '#f032e6', '#bfef45', '#fabed4', '#469990',
            '#dcbeff', '#9A6324', '#fffac8', '#800000', '#aaffc3',
            '#808000', '#ffd8b1', '#000075', '#a9a9a9', '#000000'
        ]

        color_dict = dict(zip(top_categories.index, distinct_colors[:len(top_categories)]))
        if len(unique_values) > 20:
            color_dict['Others'] = '#808080'

        return color_dict, top_categories

def update_map(column):
    # Create base map centered on mean coordinates
    with output:
      output.clear_output(wait=True)
      df_cleaned = df.dropna(subset=[column])
      m = folium.Map(
          location=[df_cleaned['_বাড়ির অবস্থান _latitude'].mean(),
                  df_cleaned['_বাড়ির অবস্থান _longitude'].mean()],
          zoom_start=12,
          prefer_canvas=True
      )
      # Add GeoJSON layer
      style_function = lambda x: {
          'fillColor': 'none',
          'color': 'black',
          'weight': 1,
          'fillOpacity': 0
      }

      tooltip = folium.GeoJsonTooltip(
          fields=['tv_name'],
          aliases=['Village:'],
          style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
      )

      geojson_layer = folium.GeoJson(
          geojson_data,
          name='Village Boundaries',
          style_function=style_function,
          tooltip=tooltip
      )
      geojson_layer.add_to(m)

      # Create color map
      if pd.api.types.is_numeric_dtype(df_cleaned[column]):
          color_map = create_color_map(df_cleaned[column])
          top_categories = None  # Not needed for numeric data
      else:
          color_map, top_categories = create_color_map(df_cleaned[column])

      # Add points to map
      for idx, row in df_cleaned.iterrows():
          if pd.api.types.is_numeric_dtype(df_cleaned[column]):
              color = color_map(row[column])
          else:
              # Use get() method with 'Others' as default for any category not in top 20
              color = color_map.get(row[column], '#808080')

          popup_text = f"Location-Respondent-Submitter : {row['loc-res-sub']}<br>UUID : {row['_uuid']}<br>{column}: {row[column]}"

          folium.CircleMarker(
              location=[row['_বাড়ির অবস্থান _latitude'],
                      row['_বাড়ির অবস্থান _longitude']],
              radius=5,
              popup=popup_text,
              color=color,
              fill=True
          ).add_to(m)

      # Add color map to the map if numeric
      if pd.api.types.is_numeric_dtype(df_cleaned[column]):
          # Create a custom legend for numeric data
          legend_html = f'''
              <div style="position: fixed;
                          bottom: 50px; right: 50px;
                          width: 200px;
                          border:2px solid grey;
                          z-index:9999;
                          background-color:white;
                          padding:10px;
                          font-size:12px;">
              <p style="font-size:14px"><b>{column}</b></p>
              '''

          # Add the numeric ranges to the legend
          for i in range(len(color_map.index)-1):
              start = color_map.index[i]
              end = color_map.index[i+1]
              color = rgba_to_hex(color_map.colors[i])
              legend_html += f'<p><span style="color:{color};">●</span> {start:.2f} - {end:.2f}</p>'

          legend_html += '</div>'
          m.get_root().html.add_child(folium.Element(legend_html))
          color_map.add_to(m)
      else:
          # Add legend for categorical data
          legend_html = f'''
              <div style="position: fixed;
                          bottom: 50px; right: 50px; width: 200px; max-height: 500px;
                          border:2px solid grey; z-index:9999; background-color:white;
                          padding:10px; overflow-y: auto; font-size:12px;">
              <p style="font-size:14px"><b>{column}</b></p>
              <p style="font-size:10px">Showing top 20 categories by frequency</p>
              '''

          # Add top categories with counts
          for value, count in top_categories.items():
              color = color_map[value]
              legend_html += f'<p><span style="color:{color};">●</span> {value} ({count})</p>'

          # Add 'Others' category if there are more categories
          if len(df_cleaned[column].unique()) > 20:
              others_count = df_cleaned[column].value_counts().iloc[20:].sum()
              legend_html += f'<p><span style="color:#808080;">●</span> Others ({others_count})</p>'

          legend_html += '</div>'
          m.get_root().html.add_child(folium.Element(legend_html))

      # Add layer control
      folium.TileLayer('OpenStreetMap').add_to(m)
      folium.LayerControl().add_to(m)

      display(m)

# Create dropdown widget
dropdown = widgets.Dropdown(
    options=plot_columns,
    description='Color by:',
    style={'description_width': 'initial'}
)

# Connect the dropdown to the function
# dropdown.observe( update_map,names='value')

widgets.interactive(update_map, column=dropdown)
display(widgets.VBox([dropdown]))
display(widgets.VBox([output]))
update_map(dropdown.value)

In [ ]:
# @title Create Scatter Plot

alt.renderers.enable('colab', embed_options={'renderer': 'svg'})
warnings.simplefilter(action='ignore', category=FutureWarning)
# Get numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Create two dropdown widgets
dropdown1 = widgets.Dropdown(
    options=numeric_cols,
    description='X-axis:',
    style={'description_width': 'initial'}
)

dropdown2 = widgets.Dropdown(
    options=numeric_cols,
    description='Y-axis:',
    style={'description_width': 'initial'}
)

# Create an Output widget to display the plot
output = widgets.Output()

def update_scatter_plot(change):
    if change.name=='value':
        x_col = dropdown1.value
        y_col = dropdown2.value
        mask = ~(df[x_col].isna() | df[y_col].isna())
        df_clean = df[mask]
        print(len(df))
        df_clean=df_clean.head(5000)

        with output:
            output.clear_output(wait=True)

            # Remove NaN values

            interval = alt.selection_interval()

            scatter=alt.Chart(df_clean, width=800, height=400).mark_circle().encode(
                x=alt.X(f'{x_col}:Q', title=f'{x_col}'),
                y=alt.Y(f'{y_col}:Q', title=f'{y_col}'),
                tooltip=[x_col,y_col,'loc-res-sub','_uuid'],
                color=alt.condition(
                    alt.datum['গ্রামের নাম'] != None,  # Check if the color column is not None
                    alt.Color('গ্রামের নাম:N', title='Color'),  # Use the color column
                    alt.value('lightgray')  # Default color for missing values
                )
            ).add_params(
                interval
            )
            ranked_text = alt.Chart(df_clean).mark_text(align='right').encode(
                y=alt.Y('row_number:O').axis(None)
            ).transform_filter(
                interval
            ).transform_window(
                row_number='row_number()'
            ).transform_filter(
                alt.datum.row_number < 50
            )

            # Data Tables
            colx = ranked_text.encode(text=x_col).properties(
                title=alt.Title(text=x_col, align='right')
            )
            coly = ranked_text.encode(text=y_col).properties(
                title=alt.Title(text=y_col, align='right')
            )
            locressub = ranked_text.encode(text='loc-res-sub').properties(
                title=alt.Title(text='loc-res-sub', align='right')
            )
            uuid=ranked_text.encode(text='_uuid').properties(
                title=alt.Title(text='id', align='right')
            )

            # display(scatter)
            fullg=alt.hconcat(
            scatter,
            alt.hconcat(uuid,colx, coly,locressub)
            ).resolve_legend(
            color="independent"
            ).configure_view(
            stroke=None
            )
            display(fullg)

dropdown2.observe(update_scatter_plot)

# Display the widgets and output
display(widgets.VBox([dropdown1, dropdown2, output]))

In [4]:
# @title Create bar chart
import time
# Get categorical and numeric columns
categorical_cols = df.select_dtypes(exclude=['int64', 'float64']).columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Create dropdown widgets
cat_dropdown = widgets.Dropdown(
    options=categorical_cols,
    description='Group by:',
    style={'description_width': 'initial'}
)

num_dropdown = widgets.Dropdown(
    options=numeric_cols,
    description='Average of:',
    style={'description_width': 'initial'}
)

# Create an Output widget to display the plot
output = widgets.Output()

def update_plot(change):
    cat_col = cat_dropdown.value
    num_col = num_dropdown.value

    with output:
        output.clear_output()
        print("Creating Chart")
        mask = ~(df[cat_col].isna() | df[num_col].isna())
        df_clean = df[mask]
        start_time = time.time()
        df_agg = df_clean.groupby(cat_col)[num_col].agg(['sum','mean', 'count']).reset_index()
        df_agg['mean'] = df_agg['mean'].round(2)
        df_agg = df_agg[df_agg['count'] >= 5]
        df_agg = df_agg.sort_values('mean', ascending=True)
        bar= alt.Chart(df_agg, width=800, height=400).mark_bar().encode(
            x=alt.X('mean',title=f'Avg  {num_col}'),
            y=cat_col,
            tooltip=[cat_col,'mean','count'],
        ).interactive()
        display(bar)



# Connect the update function to both dropdowns
cat_dropdown.observe(update_plot, names='value')
num_dropdown.observe(update_plot, names='value')

# Create the initial plot
update_plot(None)

# Display the widgets and output
display(widgets.VBox([cat_dropdown, num_dropdown, output]))